In [ ]:
SELECT  arquivos.[path] as path_minio
  FROM [MPES].[dbo].[documentos]
  join dbo.arquivos on documentos.id_arquivo_externo = arquivos.id or documentos.id_arquivo_renderizado = arquivos.id
  where id_tipo_documento = 59 AND ativo = 1 AND cancelado = 0

In [4]:
import logging
import os
import platform
import re
import shutil
import time
from datetime import datetime
from pathlib import Path # Usar pathlib para manipulação de caminhos
from typing import List, Dict, Any, Optional
import tempfile # Para arquivos temporários

import fitz  # PyMuPDF
import pytesseract
from PIL import Image
from dotenv import load_dotenv
from elasticsearch import Elasticsearch, ConnectionError as ESConnectionError
from elasticsearch.helpers import bulk
import pyodbc


# Carrega as variáveis de ambiente do .env (se existir)
load_dotenv()

# Configuração básica de logging
# Adicionar worker_id ao formato de log pode ser útil
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(processName)s - %(filename)s:%(lineno)d - %(message)s'
)
logger = logging.getLogger(__name__)

# --- Classes de Configuração (melhor que variáveis globais soltas) ---
class AppConfig:
    SQL_SERVER_CNXN_STR: Optional[str] = os.getenv('SQL_SERVER_CNXN_STR')
    ELASTICSEARCH_HOSTS: List[str] = (os.getenv('ELASTICSEARCH_HOSTS') or 'http://localhost:9200').split(',')
    ELASTICSEARCH_USER: Optional[str] = os.getenv('ELASTICSEARCH_USER')
    ELASTICSEARCH_PWD: Optional[str] = os.getenv('ELASTICSEARCH_PWD')
    
    ES_INDEX_TEXT: str = os.getenv('ES_INDEX_TEXT', 'gampes_textual')
    ES_INDEX_PAGE: str = os.getenv('ES_INDEX_PAGE', 'gampes_textual_paginas')

    MINIO_ENDPOINT: str = os.getenv('MINIO_ENDPOINT')
    MINIO_ACCESS_KEY: str = os.getenv('MINIO_ACCESS_KEY')
    MINIO_SECRET_KEY: str = os.getenv('MINIO_SECRET_KEY')

    TESSERACT_TEST_IMAGE: str = os.getenv('TESSERACT_TEST_IMAGE', 'test.png') # Opcional

    # Diretório base para saída temporária, se não usar tempfile para tudo
    # OUTPUT_BASE_DIR: Path = Path(os.getenv('OUTPUT_BASE_DIR', './ocr_output'))

    @staticmethod
    def _validate_config():
        if not AppConfig.SQL_SERVER_CNXN_STR:
            logger.error("Variável de ambiente SQL_SERVER_CNXN_STR não definida.")
            raise ValueError("SQL_SERVER_CNXN_STR não configurada.")
        logger.info('String de conexão SQL Server carregada.')
        logger.info('Hosts Elasticsearch: %s', AppConfig.ELASTICSEARCH_HOSTS)
        logger.info('Configuração MinIO: %s', {"endpoint": AppConfig.MINIO_ENDPOINT})


AppConfig._validate_config()
# AppConfig.OUTPUT_BASE_DIR.mkdir(parents=True, exist_ok=True)



def get_db_connection(conn_str: str) -> pyodbc.Connection:
    try:
        return pyodbc.connect(conn_str)
    except pyodbc.Error as ex:
        sqlstate = ex.args[0]
        logger.error(f"Erro de conexão com SQL Server (SQLSTATE: {sqlstate}): {ex}")
        raise # Re-levanta para ser tratado pelo chamador


def get_minio_file_path(id_documento_mni: str, db_conn) -> str:
    # Primeira query - busca pelo arquivo renderizado
    query_renderizado = """
        SELECT a.path as caminho_minio
        FROM MPES.dbo.documentos d WITH (NOLOCK)
        LEFT JOIN MPES.dbo.arquivos a WITH (NOLOCK) ON a.id = d.id_arquivo_renderizado
        WHERE d.Id = ?
    """
    
    # Segunda query - busca pelo arquivo original (se a primeira não retornar nada)
    query_original = """
        SELECT a.path as caminho_minio
        FROM MPES.dbo.documentos d WITH (NOLOCK)
        LEFT JOIN MPES.dbo.arquivos a WITH (NOLOCK) ON a.id = d.id_arquivo_externo
        WHERE d.Id = ?
    """
    
    with db_conn.cursor() as cursor:
        # Tenta buscar o caminho do arquivo renderizado primeiro
        cursor.execute(query_renderizado, (id_documento_mni,))
        row = cursor.fetchone()
        
        # Se não encontrou ou o path é NULL, tenta buscar o caminho do arquivo original
        if not row or not row[0]:
            cursor.execute(query_original, (id_documento_mni,))
            row = cursor.fetchone()
        
        # Retorna o path se encontrou, ou None se não encontrou em nenhuma das queries
        return row[0] if row and row[0] else None
    

# if __name__ == "__main__":
#     try:
#         conn = get_db_connection(AppConfig.SQL_SERVER_CNXN_STR)
#         # Teste simples de conexão
#         with conn.cursor() as test_cursor:
#             test_cursor.execute("SELECT 1 as test")
#             test_result = test_cursor.fetchone()
#             print(f"Teste de conexão: {test_result[0] if test_result else 'falha'}")
            
#         doc_id = "9332334"
#         print(f"Testando get_minio_file_path para o documento {doc_id}...")
#         caminho = get_minio_file_path(doc_id, conn)
#         print(f"Caminho MinIO para o documento {doc_id}: {caminho}")
#     except Exception as e:
#         logger.error(f"Erro ao testar get_minio_file_path: {e}")
#         raise  # Re-lança a exceção para ver o stack trace completo


import os
import boto3
from botocore.exceptions import ClientError, NoCredentialsError
from urllib.parse import urlparse
import logging
from dotenv import load_dotenv

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Configuração do logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class MinIODownloader:
    def __init__(self, endpoint_url, access_key, secret_key):
        """
        Inicializa o cliente MinIO
        
        Args:
            endpoint_url (str): URL do servidor MinIO
            access_key (str): Chave de acesso
            secret_key (str): Chave secreta
        """
        self.endpoint_url = endpoint_url
        self.access_key = access_key
        self.secret_key = secret_key
        
        # Configurar cliente S3 para MinIO
        self.s3_client = boto3.client(
            's3',
            endpoint_url=f'https://{endpoint_url}',
            aws_access_key_id=access_key,
            aws_secret_access_key=secret_key,
            region_name='us-east-1'  # MinIO geralmente usa esta região
        )
    
    def parse_path(self, path):
        """
        Analisa o path da tabela e extrai informações do bucket e arquivo
        
        Args:
            path (str): Path no formato da tabela
            
        Returns:
            tuple: (bucket_name, object_key)
        """
        # O path parece estar no formato: uuid|tipo.documento|mime_type|extensao
        parts = path.split('|')
        
        if len(parts) < 2:
            raise ValueError(f"Formato de path inválido: {path}")
        
        uuid = parts[0]
        document_type = parts[1]
        
        # Mapear tipos de documento para buckets
        bucket_mapping = {
            'mni.documento.original': 'gampes-mni-documento-original',
            'documento.externo': 'gampes-documento-externo',
            'documento.assinatura': 'gampes-documento-assinatura',
            'documento.renderizado': 'gampes-documento-renderizado',
            'documento.sumarizado': 'gampes-documento-sumarizado',
            'documento.transcricao': 'gampes-documento-transcricao',
            'documento.visualizacao': 'gampes-documento-visualizacao',
            'documento.pessoal': 'gampes-documento-pessoal',
            'autos.movimento': 'gampes-autos-movimento',
            'atividade.nao.procedimental': 'gampes-atividade-nao-procedimental',
            'documento.gerador.denuncia': 'gampes-documento-gerador-denuncia',
            'mni.comprovante': 'gampes-mni-comprovante',
            'mni.documento.renderizado': 'gampes-mni-documento-renderizado'
        }
        
        bucket_name = bucket_mapping.get(document_type)
        if not bucket_name:
            # Fallback: tentar inferir o bucket baseado no tipo
            logger.warning(f"Tipo de documento não mapeado: {document_type}")
            bucket_name = f"gampes-{document_type.replace('.', '-')}"
        
        # O nome do objeto no MinIO inclui a extensão
        if len(parts) >= 4:
            extension = parts[3]
            # Adiciona o ponto se não houver
            if not extension.startswith('.'):
                extension = f".{extension}"
            object_key = f"{uuid}{extension}"
        else:
            object_key = uuid
        
        return bucket_name, object_key
    
    def download_document(self, path, local_directory="downloads", filename=None):
        """
        Baixa um documento do MinIO
        
        Args:
            path (str): Path do documento na tabela
            local_directory (str): Diretório local para salvar
            filename (str): Nome do arquivo (opcional)
            
        Returns:
            str: Caminho completo do arquivo baixado
        """
        try:
            # Analisar o path
            bucket_name, object_key = self.parse_path(path)
            uuid = path.split('|')[0]
            
            # Criar diretório se não existir
            os.makedirs(local_directory, exist_ok=True)
            
            # Definir nome do arquivo
            if not filename:
                filename = object_key
            
            local_path = os.path.join(local_directory, filename)
            
            # Verificar se o objeto existe
            try:
                self.s3_client.head_object(Bucket=bucket_name, Key=object_key)
            except ClientError as e:
                if e.response['Error']['Code'] == '404':
                    # Tentar encontrar o objeto no bucket
                    found_object = self.find_object_in_bucket(bucket_name, uuid)
                    if found_object:
                        object_key = found_object
                        # Atualizar o nome do arquivo local
                        if not filename or filename == f"{uuid}.pdf":
                            filename = found_object
                            local_path = os.path.join(local_directory, filename)
                        logger.info(f"Objeto encontrado: {bucket_name}/{object_key}")
                    else:
                        logger.error(f"Objeto não encontrado no bucket {bucket_name} para UUID: {uuid}")
                        return None
                else:
                    raise
            
            # Baixar o arquivo
            logger.info(f"Baixando {bucket_name}/{object_key} -> {local_path}")
            self.s3_client.download_file(bucket_name, object_key, local_path)
            
            logger.info(f"Download concluído: {local_path}")
            return local_path
            
        except ClientError as e:
            logger.error(f"Erro ao baixar arquivo: {e}")
            return None
        except Exception as e:
            logger.error(f"Erro inesperado: {e}")
            return None
    
    def download_multiple_documents(self, documents_data, local_directory="downloads"):
        """
        Baixa múltiplos documentos
        
        Args:
            documents_data (list): Lista de dicionários com dados dos documentos
            local_directory (str): Diretório local para salvar
            
        Returns:
            list: Lista de caminhos dos arquivos baixados com sucesso
        """
        downloaded_files = []
        
        for doc in documents_data:
            try:
                path = doc['path']
                doc_id = doc.get('id', 'unknown')
                
                # Criar nome de arquivo com ID
                parts = path.split('|')
                if len(parts) >= 4:
                    extension = parts[3]
                    filename = f"{doc_id}_{parts[0]}{extension}"
                else:
                    filename = f"{doc_id}_{parts[0]}"
                
                local_path = self.download_document(path, local_directory, filename)
                if local_path:
                    downloaded_files.append(local_path)
                    
            except Exception as e:
                logger.error(f"Erro ao processar documento {doc}: {e}")
                continue
        
        return downloaded_files
    
    def find_object_in_bucket(self, bucket_name, uuid):
        """
        Procura um objeto no bucket pelo UUID, testando diferentes variações
        
        Args:
            bucket_name (str): Nome do bucket
            uuid (str): UUID do documento
            
        Returns:
            str: Nome do objeto encontrado ou None
        """
        try:
            # Listar todos os objetos do bucket
            response = self.s3_client.list_objects_v2(Bucket=bucket_name)
            
            if 'Contents' not in response:
                return None
            
            objects = [obj['Key'] for obj in response['Contents']]
            
            # Procurar por diferentes variações
            candidates = [
                uuid,                    # UUID simples
                f"{uuid}.pdf",          # UUID com .pdf
                f"{uuid}.PDF",          # UUID com .PDF
                f"{uuid}.doc",          # UUID com .doc
                f"{uuid}.docx",         # UUID com .docx
            ]
            
            for candidate in candidates:
                if candidate in objects:
                    return candidate
            
            # Se não encontrou exato, procurar por substring
            for obj in objects:
                if uuid in obj:
                    logger.info(f"Encontrado objeto similar: {obj}")
                    return obj
            
            return None
            
        except Exception as e:
            logger.error(f"Erro ao procurar objeto no bucket {bucket_name}: {e}")
            return None

    def list_buckets(self):
        """
        Lista todos os buckets disponíveis
        
        Returns:
            list: Lista de nomes dos buckets
        """
        try:
            response = self.s3_client.list_buckets()
            return [bucket['Name'] for bucket in response['Buckets']]
        except Exception as e:
            logger.error(f"Erro ao listar buckets: {e}")
            return []
    
    def list_objects(self, bucket_name, prefix=""):
        """
        Lista objetos em um bucket
        
        Args:
            bucket_name (str): Nome do bucket
            prefix (str): Prefixo para filtrar objetos
            
        Returns:
            list: Lista de objetos
        """
        try:
            response = self.s3_client.list_objects_v2(
                Bucket=bucket_name,
                Prefix=prefix
            )
            
            if 'Contents' in response:
                return [obj['Key'] for obj in response['Contents']]
            else:
                return []
                
        except Exception as e:
            logger.error(f"Erro ao listar objetos do bucket {bucket_name}: {e}")
            return []

def baixar_documento_minio(endpoint, access_key, secret_key, document_path, local_directory="downloads"):
    """
    Baixa um documento do MinIO usando as credenciais e endpoint fornecidos.

    Args:
        endpoint (str): Endpoint do MinIO (ex: 'http://localhost:9000')
        access_key (str): Chave de acesso do MinIO
        secret_key (str): Chave secreta do MinIO
        document_path (str): Path do documento na tabela
        local_directory (str): Diretório local para salvar o arquivo

    Returns:
        str: Caminho do arquivo baixado ou None se falhar
    """
    downloader = MinIODownloader(endpoint, access_key, secret_key)
    return downloader.download_document(document_path, local_directory)


def save_file_from_minio(
    file_id: str, 
    db_conn: pyodbc.Connection,
    local_directory="downloads"
) -> Optional[Path]:
    """
    Busca o caminho do arquivo no MinIO a partir do banco de dados usando o file_id,
    faz o download do documento do MinIO e retorna o caminho local do arquivo baixado.

    Parâmetros:
        file_id (str): ID do documento a ser recuperado.
        db_conn (pyodbc.Connection): Conexão ativa com o banco de dados SQL Server.

    Retorna:
        Optional[Path]: Caminho local do arquivo baixado, ou None em caso de erro.
    """
    endpoint = AppConfig.MINIO_ENDPOINT
    access_key = AppConfig.MINIO_ACCESS_KEY
    secret_key = AppConfig.MINIO_SECRET_KEY

    try:
        document_path = get_minio_file_path(file_id, db_conn)
        logger.info(f"Caminho MinIO para o documento {file_id}: {document_path}")
    except Exception as e:
        logger.error(f"Erro ao recuperar caminho do MinIO para o documento {file_id}: {e}")
        return None

    try:
        downloader = MinIODownloader(endpoint, access_key, secret_key)
        result = downloader.download_document(document_path, local_directory)
        if result:
            logger.info(f"Documento {file_id} baixado com sucesso: {result}")
            return result
        else:
            logger.error(f"Falha ao baixar documento {file_id} do MinIO.")
            return None
    except Exception as e:
        logger.error(f"Erro ao baixar documento {file_id} do MinIO: {e}")
        return None


# if __name__ == "__main__":
#     result = save_file_from_minio(
#         file_id="9332334",
#         db_conn=get_db_connection(AppConfig.SQL_SERVER_CNXN_STR)
#     )
#     print(result)


"""
# Exemplo de uso
if __name__ == "__main__":
    # Configurações do MinIO
    MINIO_ENDPOINT: str = os.getenv('MINIO_ENDPOINT')
    MINIO_ACCESS_KEY: str = os.getenv('MINIO_ACCESS_KEY')
    MINIO_SECRET_KEY: str = os.getenv('MINIO_SECRET_KEY')
    
    # Inicializar downloader
    downloader = MinIODownloader(MINIO_ENDPOINT, MINIO_ACCESS_KEY, MINIO_SECRET_KEY)
    
    # Exemplo 1: Baixar um documento específico
    try:
        conn = get_db_connection(AppConfig.SQL_SERVER_CNXN_STR)
        doc_id = "29033388"
        caminho = get_minio_file_path(doc_id, conn)
        print(f"Caminho MinIO para o documento {doc_id}: {caminho}")
    except Exception as e:
        logger.error(f"Erro ao recuperar path get_minio_file_path: {e}")

    document_path = get_minio_file_path(doc_id, conn)

    result = downloader.download_document(document_path)
    
    if result:
        print(f"Documento baixado com sucesso: {result}")
    else:
        print("Falha ao baixar documento")

    # Exemplo 3: Listar buckets
    buckets = downloader.list_buckets()
    print(f"Buckets disponíveis: {buckets}")
    
    # Exemplo 4: Listar objetos em um bucket específico
    objects = downloader.list_objects("gampes-documento-externo")
    print(f"Objetos em gampes-documento-externo: {objects[:5]}")  # Mostrar apenas os primeiros 5

"""


2025-09-29 18:26:25,285 - INFO - MainProcess - 67772466.py:56 - String de conexão SQL Server carregada.
2025-09-29 18:26:25,287 - INFO - MainProcess - 67772466.py:57 - Hosts Elasticsearch: ['http://elasticm01.mpes.gov.br:9200', 'http://elasticw01.mpes.gov.br:9200', 'http://elasticw02.mpes.gov.br:9200']
2025-09-29 18:26:25,288 - INFO - MainProcess - 67772466.py:58 - Configuração MinIO: {'endpoint': 's3.mpes.mp.br:9000'}


'\n# Exemplo de uso\nif __name__ == "__main__":\n    # Configurações do MinIO\n    MINIO_ENDPOINT: str = os.getenv(\'MINIO_ENDPOINT\')\n    MINIO_ACCESS_KEY: str = os.getenv(\'MINIO_ACCESS_KEY\')\n    MINIO_SECRET_KEY: str = os.getenv(\'MINIO_SECRET_KEY\')\n\n    # Inicializar downloader\n    downloader = MinIODownloader(MINIO_ENDPOINT, MINIO_ACCESS_KEY, MINIO_SECRET_KEY)\n\n    # Exemplo 1: Baixar um documento específico\n    try:\n        conn = get_db_connection(AppConfig.SQL_SERVER_CNXN_STR)\n        doc_id = "29033388"\n        caminho = get_minio_file_path(doc_id, conn)\n        print(f"Caminho MinIO para o documento {doc_id}: {caminho}")\n    except Exception as e:\n        logger.error(f"Erro ao recuperar path get_minio_file_path: {e}")\n\n    document_path = get_minio_file_path(doc_id, conn)\n\n    result = downloader.download_document(document_path)\n\n    if result:\n        print(f"Documento baixado com sucesso: {result}")\n    else:\n        print("Falha ao baixar document

In [5]:
def baixar_documentos_por_tipo(tipo_documento=59, local_directory="downloads"):
    """
    Baixa todos os documentos de um tipo específico do MinIO
    
    Args:
        tipo_documento (int): ID do tipo de documento (padrão: 59)
        local_directory (str): Diretório local para salvar os arquivos
        
    Returns:
        list: Lista de caminhos dos arquivos baixados com sucesso
    """
    try:
        # Estabelecer conexão com o banco de dados
        conn = get_db_connection(AppConfig.SQL_SERVER_CNXN_STR)
        logger.info(f"Conexão com banco de dados estabelecida")
        
        # Executar a query para obter a lista de paths
        query = """
        SELECT arquivos.[path] as path_minio
        FROM [MPES].[dbo].[documentos]
        JOIN dbo.arquivos ON documentos.id_arquivo_externo = arquivos.id 
            OR documentos.id_arquivo_renderizado = arquivos.id
        WHERE id_tipo_documento = ? AND ativo = 1 AND cancelado = 0
        """
        
        with conn.cursor() as cursor:
            cursor.execute(query, (tipo_documento,))
            resultados = cursor.fetchall()
            
        logger.info(f"Encontrados {len(resultados)} documentos do tipo {tipo_documento}")
        
        if not resultados:
            logger.warning(f"Nenhum documento encontrado para o tipo {tipo_documento}")
            return []
        
        # Inicializar o downloader do MinIO
        downloader = MinIODownloader(
            AppConfig.MINIO_ENDPOINT,
            AppConfig.MINIO_ACCESS_KEY,
            AppConfig.MINIO_SECRET_KEY
        )
        
        # Preparar lista de documentos para download
        documentos_para_baixar = []
        for i, row in enumerate(resultados):
            path_minio = row[0]
            if path_minio:  # Só processar se o path não for nulo
                documentos_para_baixar.append({
                    'id': f"doc_{tipo_documento}_{i}",
                    'path': path_minio
                })
                logger.debug(f"Documento {i}: {path_minio}")
        
        logger.info(f"Preparados {len(documentos_para_baixar)} documentos para download")
        
        # Baixar os documentos
        arquivos_baixados = downloader.download_multiple_documents(
            documentos_para_baixar, 
            local_directory
        )
        
        logger.info(f"Download concluído: {len(arquivos_baixados)} arquivos baixados com sucesso")
        
        # Fechar conexão com o banco
        conn.close()
        
        return arquivos_baixados
        
    except pyodbc.Error as e:
        logger.error(f"Erro de banco de dados: {e}")
        return []
    except Exception as e:
        logger.error(f"Erro inesperado: {e}")
        return []


def baixar_documentos_por_tipo_com_detalhes(tipo_documento=59, local_directory="downloads"):
    """
    Versão alternativa que inclui mais detalhes sobre cada documento
    
    Args:
        tipo_documento (int): ID do tipo de documento
        local_directory (str): Diretório local para salvar os arquivos
        
    Returns:
        dict: Estatísticas e lista de resultados detalhados
    """
    try:
        conn = get_db_connection(AppConfig.SQL_SERVER_CNXN_STR)
        
        # Query expandida para incluir mais informações
        query = """
        SELECT 
            d.Id as id_documento,
            arquivos.[path] as path_minio,
            d.id_tipo_documento,
            d.ativo,
            d.cancelado
        FROM [MPES].[dbo].[documentos] d
        JOIN dbo.arquivos ON d.id_arquivo_externo = arquivos.id 
            OR d.id_arquivo_renderizado = arquivos.id
        WHERE d.id_tipo_documento = ? AND d.ativo = 1 AND d.cancelado = 0
        """
        
        with conn.cursor() as cursor:
            cursor.execute(query, (tipo_documento,))
            colunas = [column[0] for column in cursor.description]
            resultados = cursor.fetchall()
        
        logger.info(f"Encontrados {len(resultados)} documentos do tipo {tipo_documento}")
        
        if not resultados:
            return {
                'sucesso': False,
                'mensagem': f'Nenhum documento encontrado para o tipo {tipo_documento}',
                'total': 0,
                'baixados': 0,
                'arquivos': []
            }
        
        # Converter resultados para dicionários
        documentos = []
        for row in resultados:
            doc_dict = dict(zip(colunas, row))
            documentos.append(doc_dict)
        
        # Inicializar downloader
        downloader = MinIODownloader(
            AppConfig.MINIO_ENDPOINT,
            AppConfig.MINIO_ACCESS_KEY,
            AppConfig.MINIO_SECRET_KEY
        )
        
        # Preparar lista para download
        documentos_para_baixar = []
        for doc in documentos:
            if doc['path_minio']:
                documentos_para_baixar.append({
                    'id': f"doc_{doc['id_documento']}",
                    'path': doc['path_minio']
                })
        
        # Realizar download
        arquivos_baixados = downloader.download_multiple_documents(
            documentos_para_baixar, 
            local_directory
        )
        
        # Preparar resultado final
        resultado = {
            'sucesso': True,
            'total_documentos': len(documentos),
            'total_para_download': len(documentos_para_baixar),
            'baixados_com_sucesso': len(arquivos_baixados),
            'arquivos_baixados': arquivos_baixados,
            'documentos': documentos
        }
        
        conn.close()
        return resultado
        
    except Exception as e:
        logger.error(f"Erro ao baixar documentos: {e}")
        return {
            'sucesso': False,
            'erro': str(e),
            'total': 0,
            'baixados': 0,
            'arquivos': []
        }


# Função principal para executar o download
def main():
    """
    Função principal para executar o download dos documentos
    """
    try:
        logger.info("Iniciando download de documentos do tipo 59")
        
        # Usar a função básica
        arquivos_baixados = baixar_documentos_por_tipo(
            tipo_documento=59, 
            local_directory="documentos_baixados"
        )
        
        # Ou usar a função detalhada para mais informações
        # resultado = baixar_documentos_por_tipo_com_detalhes(
        #     tipo_documento=59,
        #     local_directory="documentos_baixados"
        # )
        
        logger.info(f"Processo concluído. {len(arquivos_baixados)} arquivos baixados.")
        
        # Listar arquivos baixados
        for i, arquivo in enumerate(arquivos_baixados, 1):
            logger.info(f"{i}. {arquivo}")
            
    except Exception as e:
        logger.error(f"Erro na execução principal: {e}")


if __name__ == "__main__":
    # Executar o download
    main()
    
    # Exemplo adicional: baixar tipos específicos de documento
    # tipos_documento = [59, 60, 61]  # Lista de tipos que você quer baixar
    
    # for tipo in tipos_documento:
    #     logger.info(f"Baixando documentos do tipo {tipo}")
    #     resultado = baixar_documentos_por_tipo(tipo_documento=tipo)
    #     logger.info(f"Tipo {tipo}: {len(resultado)} arquivos baixados")

2025-09-29 18:26:30,289 - INFO - MainProcess - 373929420.py:179 - Iniciando download de documentos do tipo 59
2025-09-29 18:26:30,747 - INFO - MainProcess - 373929420.py:15 - Conexão com banco de dados estabelecida
2025-09-29 18:26:31,779 - INFO - MainProcess - 373929420.py:30 - Encontrados 846 documentos do tipo 59
2025-09-29 18:26:32,685 - INFO - MainProcess - 373929420.py:54 - Preparados 846 documentos para download
2025-09-29 18:26:33,226 - INFO - MainProcess - 67772466.py:262 - Baixando gampes-documento-externo/6197a3e3-29ab-4436-a7ff-03e7ce63fb6a.pdf -> documentos_baixados\doc_59_0_6197a3e3-29ab-4436-a7ff-03e7ce63fb6a.pdf
2025-09-29 18:26:33,270 - INFO - MainProcess - 67772466.py:265 - Download concluído: documentos_baixados\doc_59_0_6197a3e3-29ab-4436-a7ff-03e7ce63fb6a.pdf
2025-09-29 18:26:33,291 - INFO - MainProcess - 67772466.py:262 - Baixando gampes-documento-externo/78ac97c0-5d32-4cd9-8ce7-5434a2fd9048.pdf -> documentos_baixados\doc_59_1_78ac97c0-5d32-4cd9-8ce7-5434a2fd9048.